In [1]:
# !pip install mybatis_mapper2sql OpenDartReader

In [2]:
#!pip install git+https://github.com/nockchun/rspy --force
import rspy as rsp
rsp.setSystemWarning(off=True)

In [3]:
import OpenDartReader
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [4]:
api_key = '125e216fefd7c214a1a66784b73f76075dff77b7'
dart = OpenDartReader(api_key)

In [5]:
dart.find_corp_code("안랩")

'00298270'

In [6]:
dart.report(dart.find_corp_code("032830"), '증자', 2017)

,rcept_no,corp_cls,corp_code,corp_name,isu_dcrs_de,isu_dcrs_stle,isu_dcrs_stock_knd,isu_dcrs_qy,isu_dcrs_mstvdv_fval_amount,isu_dcrs_mstvdv_amount
0,20180402004606,Y,00126256,삼성생명,-,-,-,-,-,-


In [7]:
db = rsp.PGManager()
db.createPool("postgres", "postgres", "postgres", "192.168.0.200")
db.setMapper("mapper.xml")

In [8]:
df = db.queryPandas("seJemu", params=("006840"), dtype={"code":"str", "year":"int", "quarter":"int"})

In [9]:
df.columns

Index(['code', 'year', 'quarter', '자산총계', '부채총계', '자본총계', '매출액', '매출총이익',
       '영업이익', '비영업손익', '매출원가', '판관비', '당기순이익', '세전계속사업손익', '잉여금', '영업활동현금흐름',
       '투자활동현금흐름', '재무활동현금흐름', '순현금흐름', 'per', 'pbr', 'cfps', 'eps', 'bps',
       'pcr', '지배주주순이익', 'sps', 'ebitda', 'ev_ebitda', 'psr', '유동비율', '부채비율',
       '자기자본비율', '이자보상비율', '유보율', '매출액증가율', '총자산증가율', '지배주주eps증가율', '판관비증가율',
       '영업이익증가율', 'ebitda증가율', '매출액총이익률', '예대마진율', 'ebitda마진율', '지배주주roe',
       '세전계속사업이익률', '영업이익률', 'roa', '순사업비율', '대출채권회전율'],
      dtype='object')

In [10]:
buySeasonByQuater = {
    1:["-07-01", "-07-30"],
    2:["-10-01", "-10-30"],
    3:["-01-01", "-01-30"],
    4:["-04-01", "-04-20"]
}

In [11]:
import queue
import pandas_profiling as pp

In [12]:
# pp.ProfileReport(df, title="Hello")

In [13]:
class TimeSeriesAnalyzer():
    def __init__(self, dataframe, timeDict):
        self.__df = dataframe
        self.__timeDict = timeDict
        self.__timeDictKeys = list(timeDict.keys())
        self.__buckets = []
    
    def __dicLoop(self, df, idx=0):
        key = self.__timeDictKeys[idx]
        value = self.__timeDict.get(key)
        
        for item in value:
            dfCurrent = df[df[key] == item]
            if len(self.__timeDictKeys)-1 > idx:
                nextIdx = idx+1
                self.__dicLoop(dfCurrent, nextIdx)
            else:
                for bucket in self.__buckets:
                    bucket.add(dfCurrent)
                        
    def addBucket(self, bucket):
        self.__buckets.append(bucket)
    
    def analysis(self):
        self.__dicLoop(self.__df)

In [14]:
class TimeBucket():
    def __init__(self, name, seriesLength=3):
        self.__name = name
        self.__seriesLength = seriesLength
        self.__queue = queue.Queue()
        self.__simulPrice = 0
    
    def getName(self):
        return self.__name
    
    def add(self, data):
        val = data[self.__name].to_numpy()
        if len(val) == 1:
            self.__queue.put(val[0])
        else:
            self.__queue.put(0)
        
        if self.__queue.qsize() > self.__seriesLength:
            self.__queue.get()
        if self.__queue.qsize() == self.__seriesLength:
            self.__chkIncrease(data)
    
    def __chkIncrease(self, data):
        temp = np.array(self.__queue.queue)
        tempf = np.concatenate((np.array([0]), temp[:-1]))
        
        try:
            if all((temp[1:] - temp[:-1]) < 0):
                values = data[["code", "year", "quarter"]].values[0]
                code, year, quarter = values
                print(f"[+] {code, year, quarter} : {self.__name}/{self.__seriesLength} >> {self.__queue.queue}")

                if quarter > 2:
                    year += 1
                priceCloseDiff = db.query("sePriceCloseDiff", params=(code, str(year)+buySeasonByQuater[quarter][0], str(year)+buySeasonByQuater[quarter][1]))[0]
                self.__simulPrice += priceCloseDiff[1]*10 - priceCloseDiff[0]*10
        except:
            pass
    
    def getSimulPrice(self):
        return self.__simulPrice
    
    def __repr__(self):
        return f"{self.__name}/{self.__seriesLength} >> {self.__queue.queue}"

In [15]:
totalSimPrice = 0
def simul(code):
    global totalSimPrice
    df = db.queryPandas("seJemu", params=(code), dtype={"code":"str", "year":"int", "quarter":"int"})
    tsa = TimeSeriesAnalyzer(df, {"year":range(2010, 2020), "quarter":[4]})
    tb = TimeBucket("pbr", 5)
    tsa.addBucket(tb)
    tsa.analysis()
    totalSimPrice += tb.getSimulPrice()

In [16]:
totalSimPrice

0

In [17]:
codes = pd.read_excel("data/거래소코드.xlsx", dtype={"코드":str}, sheet_name="소형")
codes = codes.to_numpy()

In [18]:
df.columns

Index(['code', 'year', 'quarter', '자산총계', '부채총계', '자본총계', '매출액', '매출총이익',
       '영업이익', '비영업손익', '매출원가', '판관비', '당기순이익', '세전계속사업손익', '잉여금', '영업활동현금흐름',
       '투자활동현금흐름', '재무활동현금흐름', '순현금흐름', 'per', 'pbr', 'cfps', 'eps', 'bps',
       'pcr', '지배주주순이익', 'sps', 'ebitda', 'ev_ebitda', 'psr', '유동비율', '부채비율',
       '자기자본비율', '이자보상비율', '유보율', '매출액증가율', '총자산증가율', '지배주주eps증가율', '판관비증가율',
       '영업이익증가율', 'ebitda증가율', '매출액총이익률', '예대마진율', 'ebitda마진율', '지배주주roe',
       '세전계속사업이익률', '영업이익률', 'roa', '순사업비율', '대출채권회전율'],
      dtype='object')

In [19]:
for code, name in codes:
    simul(code)

[+] ('001460', 2019, 4) : pbr/5 >> deque([1.02, 0.85, 0.64, 0.51, 0.46])
[+] ('000590', 2018, 4) : pbr/5 >> deque([0.5, 0.45, 0.44, 0.43, 0.34])
[+] ('000590', 2019, 4) : pbr/5 >> deque([0.45, 0.44, 0.43, 0.34, 0.3])
[+] ('016610', 2014, 4) : pbr/5 >> deque([0.5, 0.46, 0.34, 0.27, 0.25])
[+] ('004840', 2018, 4) : pbr/5 >> deque([0.89, 0.8, 0.7, 0.45, 0.38])
[+] ('004840', 2019, 4) : pbr/5 >> deque([0.8, 0.7, 0.45, 0.38, 0.29])
[+] ('082740', 2015, 4) : pbr/5 >> deque([0.99, 0.56, 0.54, 0.44, 0.27])
[+] ('009440', 2018, 4) : pbr/5 >> deque([1.61, 1.32, 1.06, 0.71, 0.47])
[+] ('058860', 2019, 4) : pbr/5 >> deque([1.04, 0.8, 0.64, 0.52, 0.47])
[+] ('000680', 2017, 4) : pbr/5 >> deque([0.48, 0.41, 0.4, 0.38, 0.35])
[+] ('000680', 2018, 4) : pbr/5 >> deque([0.41, 0.4, 0.38, 0.35, 0.29])
[+] ('004250', 2019, 4) : pbr/5 >> deque([1.56, 1.1, 0.87, 0.61, 0.56])
[+] ('003570', 2019, 4) : pbr/5 >> deque([0.63, 0.45, 0.38, 0.36, 0.33])
[+] ('101060', 2016, 4) : pbr/5 >> deque([1.98, 1.14, 0.85, 0.

In [20]:
f"{totalSimPrice:,}"

'437,050'